In [ ]:
import importlib

# List of required packages
packages = ['matplotlib', 'plotly', 'pandas', 'numpy', 'datetime', 'seaborn', 'scikit-learn', 'geopandas', 'shapely', 'pyshp']

# Check if packages are installed
missing_packages = [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]

# Install missing packages
if missing_packages:
    %pip install {' '.join(missing_packages)}

# Import the packages
import pandas as pd
import numpy as np
import datetime
import matplotlib
from matplotlib import pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import re
import geopandas as gpd
import shapefile as shp
from shapely.geometry import Point

In [ ]:
def clearvars():    
    for el in sorted(globals()):
        if '__' not in el:
                print(f'deleted: {el}')
                del el
clearvars()

In [ ]:
base_path = 'dataRaw/'
dataRaw = pd.read_csv(base_path + 'data.csv', sep=',', low_memory=False)
holidays_eventsRaw = pd.read_csv(base_path + 'holidays_events(India).csv', sep=';')
itemsRaw = pd.read_csv(base_path + 'items.csv', sep=';', low_memory=False)
oilRaw = pd.read_csv(base_path + 'oil(India).csv', sep=',', skipinitialspace=True)
storesRaw = pd.read_csv(base_path + 'stores.csv', sep=';', encoding='ISO-8859-1')
testRaw = pd.read_csv(base_path + 'test.csv', sep=',')
transactionsRaw = pd.read_csv(base_path + 'transactions.csv', sep=',')


In [ ]:
test = testRaw.copy()
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')

# --------------------------------------------------------------------------------------------

""" test.to_csv('data/test.csv', index=False)
> git push origin main:main
remote: error: Trace: 25213f19d457fb2989564964b10ab27c2fb682233e594fb81ff2ed78998776c2        
remote: error: See https://gh.io/lfs for more information.        
remote: error: File data/test.csv is 120.32 MB; this exceeds GitHub's file size limit of 100.00 MB        
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/florboydens/Data-Analytics-Case-2.git
 ! [remote rejected] main -> main (pre-receive hook declined)
error: failed to push some refs to 'https://github.com/florboydens/Data-Analytics-Case-2.git' """

In [ ]:
data = dataRaw.copy()
data['date'] = pd.to_datetime(dataRaw['date'], format='%Y-%m-%d')
data['unit_sales'] = dataRaw['unit_sales'].round(2)
data['onpromotion'].fillna(False, inplace=True)

# --------------------------------------------------------------------------------------------

""" 
data.to_csv('data/data.csv', index=False)
...
Zelfde reden als de Test file, te groot
"""

In [ ]:
mask = oilRaw['dcoilwtico;;'].str.contains('%')
oil = oilRaw[~mask].copy()
oil.reset_index(drop=True, inplace=True)
oil['dcoilwtico;;'] = oil['dcoilwtico;;'].str.replace(';', '')
oil['dcoilwtico;;'] = oil['dcoilwtico;;'].apply(lambda x: x.strip())
oil = oil[~(oil['dcoilwtico;;'] == '')]
try:
    oil['dcoilwtico;;'] = oil['dcoilwtico;;'].astype(float).round(2)
except:
    oil.dropna(subset=['dcoilwtico;;'], inplace=True)
oil.rename(columns={'dcoilwtico;;': 'dcoilwtico'}, inplace=True)

oilExtra = pd.read_csv('dataRaw/oil(India).csv', sep=';', names=['date', 'price', 'change', 'extra'], skiprows=1)
oilExtra.drop(columns=['extra'], inplace=True)
oilExtra.dropna(inplace=True)
oilExtra = oilExtra.iloc[:61].copy()
oilExtra['price'] = oilExtra['price'].str.replace(',', '').astype(float)

# --------------------------------------------------------------------------------------------

oil.to_csv('data/oil.csv', index=False)
oilExtra.to_csv('data/oil(INR).csv', index=False)

del oil, oilExtra


In [ ]:
itemsRaw = itemsRaw.dropna()
try:
    itemsRaw['item_nbr'] = itemsRaw['item_nbr'].astype(int)
except:
    pass
itemsRaw['Price'] = itemsRaw['Price'].round(2)
itemsRaw['perishable'] = itemsRaw['perishable'].map({0: False, 1: True})

# --------------------------------------------------------------------------------------------

itemsRaw.to_csv('data/items.csv', index=False)

In [ ]:
transactionsRaw['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')
transactionsRaw['month_day'] = transactionsRaw['date'].dt.strftime('%m-%d')

# --------------------------------------------------------------------------------------------

transactionsRaw.to_csv('data/transactions.csv', index=False)

del transactionsRaw

In [ ]:
holidays_eventsRaw = holidays_eventsRaw.drop(columns=['type'])
holidays_eventsRaw.dropna(subset=['date'], inplace=True)
holidays_eventsRaw['date'] = holidays_eventsRaw['date'].str.replace(' ', '/')
holidays_eventsRaw['date'] = holidays_eventsRaw['date'].str.replace('mrt', 'mar')
holidays_eventsRaw['date'] = holidays_eventsRaw['date'].str.replace('mei', 'may')
holidays_eventsRaw['date'] = holidays_eventsRaw['date'].str.replace('okt', 'oct')
holidays_eventsRaw = holidays_eventsRaw.drop_duplicates()
holidays_eventsRaw['date'] = pd.to_datetime(holidays_eventsRaw['date'], format='%d/%b')   #.dt.strftime('%Y-%m-%d')
# if the date is the same from earlier in the dataset, drop the row.
holidays_eventsRaw = holidays_eventsRaw[~holidays_eventsRaw['date'].duplicated()]
holidays_eventsRaw['month_day'] = holidays_eventsRaw['date'].dt.strftime('%m-%d')
holidays_eventsRaw = holidays_eventsRaw.sort_values('date')
holidays_eventsRaw = holidays_eventsRaw.reindex(columns=['date', 'month_day', 'description', 'transferred'])
holidays_eventsRaw.rename(columns={'transferred': 'type'}, inplace=True)

# --------------------------------------------------------------------------------------------

holidays_eventsRaw.to_csv('data/holidays_events.csv', index=False)

del holidays_eventsRaw

In [ ]:
# drop all the rows with NaN store_nbr because they are not in the rest of the data set
storesRaw.dropna(subset=['store_nbr'], inplace=True)

# --------------------------------------------------------------------------------------------

storesRaw.to_csv('data/stores.csv', index=False)

del storesRaw